In [209]:
import pandas as pd
import numpy as np

In [210]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.cross_validation import cross_val_score

In [211]:
# to test model accuracy
from sklearn import metrics
# to split data into test and train
from sklearn.cross_validation import train_test_split


In [212]:
train = pd.read_csv("https://raw.githubusercontent.com/leelaabhiram/ensemble/master/train.csv")
train.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [198]:
to_predict_data = pd.read_csv("https://raw.githubusercontent.com/leelaabhiram/ensemble/master/test.csv")
print(to_predict_data.head())
tes = testData(to_predict_data)
tes.eda()

    Loan_ID Gender Married Dependents     Education Self_Employed  \
0  LP001015   Male     Yes          0      Graduate            No   
1  LP001022   Male     Yes          1      Graduate            No   
2  LP001031   Male     Yes          2      Graduate            No   
3  LP001035   Male     Yes          2      Graduate            No   
4  LP001051   Male      No          0  Not Graduate            No   

   ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
0             5720                  0       110.0             360.0   
1             3076               1500       126.0             360.0   
2             5000               1800       208.0             360.0   
3             2340               2546       100.0             360.0   
4             3276                  0        78.0             360.0   

   Credit_History Property_Area  
0             1.0         Urban  
1             1.0         Urban  
2             1.0         Urban  
3             NaN     

In [199]:
for col in train:
    print(train[col].isnull().sum())
    print(col)

0
Loan_ID
13
Gender
3
Married
15
Dependents
0
Education
32
Self_Employed
0
ApplicantIncome
0
CoapplicantIncome
22
LoanAmount
14
Loan_Amount_Term
50
Credit_History
0
Property_Area
0
Loan_Status


In [200]:
class toPredictData(object):
    def __init__(self, pred):
        self.pred=pred
        self.length=len(self.pred)
        
    def eda(self):
        
        cols=list(self.pred)
        # converting categorical variables to numeric variables
        # select only the object type column names from the data frame
        df_cat_cols = list(self.pred.select_dtypes(include=['object']).copy())
        for k in df_cat_cols:
            if(self.pred[k].nunique()>5):
                self.pred=self.pred.drop(k,axis=1)
                df_cat_cols.remove(k)
                
        self.pred= pd.get_dummies(self.pred , columns = df_cat_cols)
        
        self.addMissingValues()
        
        
    def addMissingValues(self):
        
        for col in self.pred:
            
            if self.pred[col].isnull().sum() > 0 :
                
                mean = sum(self.pred[col][self.pred[col].notnull()])/self.length
                self.pred[col].fillna(value=mean, inplace=True)

In [216]:
class ensemble(object):
    def __init__(self,data,toPredictObject,col_name,logistic_regression=True,knn=True,gbm=True,random_forest=True):
        
        self.original_data=data
        self.data=data
        self.length=len(data)
        # col_name is the name of the column to be prdicted in the data set
        self.col_name=col_name
        # col is the whole column
        self.col=self.original_data[self.col_name].to_frame()
        self.logistic_regression=logistic_regression
        self.knn=knn
        self.gbm=gbm 
        self.random_forest=random_forest
        #global variable for trial purposes
        self.trial=None
        
        #to predict object
        self.toPred = toPredictObject
        
        #created models
        self.log_model=None
        self.knn_model=None
        
        #predicted valus list for test data
        self.log_pred_test = None
        self.knn_pred_test = None
        
        #data after splitting
        self.train_X = None
        self.test_X = None
        
        self.train_y = None
        self.test_y = None
        
    def splitData(self):
        
        self.train_X,self.test_X,self.train_y,self.test_y = train_test_split(self.data,self.col, test_size=0.3, random_state = 4)
        
    def addMissingValues(self):
        
        for col in self.data:
            
            if self.data[col].isnull().sum() > 0 :
                
                mean = sum(self.data[col][self.data[col].notnull()])/self.length
                self.data[col].fillna(value=mean, inplace=True)
            
    
    def eda(self):
        # dropping the column to be predicted
        self.data=self.data.drop(self.col_name,axis=1)
        cols=list(self.data)
        # converting categorical variables to numeric variables
        # select only the object type column names from the data frame
        df_cat_cols = list(self.data.select_dtypes(include=['object']).copy())
        for k in df_cat_cols:
            if(self.data[k].nunique()>5):
                self.data=self.data.drop(k,axis=1)
                df_cat_cols.remove(k)
        
        # 1- hot encoding for all the categorical variables with less than 5 types of unique values
        # this also removes the original categorical columns
        self.data= pd.get_dummies(self.data , columns = df_cat_cols)
        
        self.addMissingValues()
        
        # converting into series for pandas to understand its the outcome var
        self.col=np.ravel(self.col)
        pass 
    
    def log_reg_algo(self):
        
        model=LogisticRegression()
        model.fit(self.train_X,self.train_y)
        print("training data accuracy = ", model.score(self.train_X,self.train_y))
        # assigning model to global class variable
        self.log_model=model
        self.log_pred_test = model.predict(self.test_X)
        print("test data accuracy = " , model.score(self.test_X,self.test_y))
    
    def log_train_accuracy(self):
        train_pred=self.log_model.predict(x.data)
        print(metrics.accuracy_score(x.col,train_pred))
        return 0
        
    
    def knn_algo(self):
        knn = KNeighborsClassifier(n_neighbors=5)
        knn.fit(self.data, self.col)
        # assigning model to global class variable
        self.knn_model=knn
        print("train data accuracy = " , knn.score(self.train_X,self.train_y))
        self.knn_pred_test = knn.predict(self.test_X)
        print("test data accuracy = " , knn.score(self.test_X,self.test_y))
        pass
    
    def gbm_algo(self): 
        pass
    
    def rand_for_algo(self):
        pass
    
    def run(self):
        self.eda()
        self.splitData()
        if(self.logistic_regression == True):
            self.log_reg_algo()
        
        if(self.knn==True):
            self.knn_algo()
            
        if(self.gbm==True):
            self.gbm_algo()
            
        

In [215]:
x.data.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender_Female,Gender_Male,Married_No,Married_Yes,Dependents_0,Dependents_1,Dependents_2,Dependents_3+,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban
0,5849,0.0,141.166124,360.0,1.0,0,1,1,0,1,0,0,0,1,0,1,0,0,0,1
1,4583,1508.0,128.000000,360.0,1.0,0,1,0,1,0,1,0,0,1,0,1,0,1,0,0
2,3000,0.0,66.000000,360.0,1.0,0,1,0,1,1,0,0,0,1,0,0,1,0,0,1
3,2583,2358.0,120.000000,360.0,1.0,0,1,0,1,1,0,0,0,0,1,1,0,0,0,1
4,6000,0.0,141.000000,360.0,1.0,0,1,1,0,1,0,0,0,1,0,1,0,0,0,1


In [217]:
x=ensemble(train, tes, 'Loan_Status')
x.eda()
x.splitData()

In [218]:
x.knn_algo()

train data accuracy =  0.736596736597
test data accuracy =  0.713513513514


In [220]:
x.knn_pred_test

array(['Y', 'Y', 'N', 'N', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'N', 'Y',
       'N', 'N', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y',
       'N', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y',
       'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'N', 'N', 'Y', 'Y', 'N', 'Y',
       'Y', 'Y', 'Y', 'Y', 'N', 'N', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y',
       'Y', 'N', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y',
       'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y',
       'N', 'N', 'Y', 'Y', 'N', 'Y', 'N', 'Y', 'N', 'Y', 'Y', 'Y', 'Y',
       'N', 'N', 'N', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y',
       'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'N',
       'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y',
       'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'N', 'N', 'Y', 'Y',
       'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y',
       'N', 'N', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y

In [205]:
len(x.test_X)

185

In [206]:
len(x.data)

614

In [207]:
429+185

614

In [208]:
x=ensemble(train, tes, 'Loan_Status',gbm=False)
x.run()

training data accuracy =  0.815850815851
test data accuracy =  0.8
train data accuracy =  0.736596736597
test data accuracy =  0.713513513514
